In [1]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import os
import requests
from pathlib import Path
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import API data
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [3]:
# Load .env enviroment variables
load_dotenv("ProjectAPI.env")

True

In [4]:
# The original tickers plus 6 new ones to consider other golbal market, across various sectors compared to the global spy movement  

# Set the tickers
tickers = ["SPY", "IWM", "EZU", "EWU", "EWJ", "EWZ", "EWA", "EWC", "MCHI", "EWW", 'XLK','XLV','XLP','ARKK','ICLN','ROBO']

# Create the shares DataFrame
df_etf = pd.DataFrame(index=tickers)

In [5]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("Alpaca_API_Key")
alpaca_secret_key = os.getenv("Alpaca_Secret_Key")

# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [6]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [7]:
# Create current date variable
import datetime
today = datetime.date.today()
historical = today - datetime.timedelta(days=550)

In [8]:
# Format current date as ISO format
# Set start and end datetimes of 18 months
start_date = pd.Timestamp(historical, tz="America/New_York").isoformat()
end_date = pd.Timestamp(today, tz="America/New_York").isoformat()

In [14]:
# Set timeframe to 18 months for the Alpaca API
timeframe = "1Day"


In [15]:
#8
# Get 18 months of pricing data for ETF portfolio
df_portfolio = alpaca.get_bars(tickers, timeframe, start=start_date, end=end_date).df

df_portfolio

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2022-08-19 04:00:00+00:00,44.69,46.410,44.490,151243,46.2400,28454333,45.071206,ARKK
2022-08-22 04:00:00+00:00,43.61,44.790,43.280,137461,43.9600,24679569,43.775314,ARKK
2022-08-23 04:00:00+00:00,43.37,44.600,42.730,110431,43.5001,20800628,43.558235,ARKK
2022-08-24 04:00:00+00:00,44.50,45.150,43.140,88085,43.5800,17435658,44.435014,ARKK
2022-08-25 04:00:00+00:00,45.78,45.780,44.380,77794,45.0400,14731237,45.146808,ARKK
...,...,...,...,...,...,...,...,...
2024-02-13 05:00:00+00:00,142.86,144.470,142.150,96008,143.4700,11170135,142.935492,XLV
2024-02-14 05:00:00+00:00,144.10,144.150,143.075,83201,143.3500,9701053,143.630030,XLV
2024-02-15 05:00:00+00:00,145.15,145.430,143.990,90086,144.2000,9294805,144.959734,XLV


In [20]:
# Reorganize the DataFrame and separate ticker data
# Order:  SPY, EWA, EWC, EWJ, EWU, EWZ, EWW, EZU, IWM, MCHI, 'XLK','XLV','XLP','ARKK','ICLN','ROBO'

# Separate ticker data
SPY = df_portfolio[df_portfolio["symbol"]=="SPY"].drop("symbol", axis=1)
EWA = df_portfolio[df_portfolio["symbol"]=="EWA"].drop("symbol", axis=1)
EWC = df_portfolio[df_portfolio["symbol"]=="EWC"].drop("symbol", axis=1)
EWJ = df_portfolio[df_portfolio["symbol"]=="EWJ"].drop("symbol", axis=1)
EWU = df_portfolio[df_portfolio["symbol"]=="EWU"].drop("symbol", axis=1)
EWZ = df_portfolio[df_portfolio["symbol"]=="EWZ"].drop("symbol", axis=1)
EWW = df_portfolio[df_portfolio["symbol"]=="EWW"].drop("symbol", axis=1)
EZU = df_portfolio[df_portfolio["symbol"]=="EZU"].drop("symbol", axis=1)
IWM = df_portfolio[df_portfolio["symbol"]=="IWM"].drop("symbol", axis=1)
MCHI = df_portfolio[df_portfolio["symbol"]=="MCHI"].drop("symbol", axis=1)
XLK = df_portfolio[df_portfolio["symbol"]=="XLK"].drop("symbol", axis=1)
XLV = df_portfolio[df_portfolio["symbol"]=="XLV"].drop("symbol", axis=1)
XLP = df_portfolio[df_portfolio["symbol"]=="XLP"].drop("symbol", axis=1)
ARKK = df_portfolio[df_portfolio["symbol"]=="ARKK"].drop("symbol", axis=1)
ICLN = df_portfolio[df_portfolio["symbol"]=="ICLN"].drop("symbol", axis=1)
ROBO = df_portfolio[df_portfolio["symbol"]=="ROBO"].drop("symbol", axis=1)

# Concatenate the ticker DataFrames
df_portreo = pd.concat([SPY, EWA, EWC, EWJ, EWU, EWZ, EWW, EZU, IWM, MCHI, XLK, XLV, XLP, ARKK, ICLN, ROBO], axis=1)

# Rename columns
df_portreo.columns=['SPY_close', 'EWA_close', 'EWC_close', 'EWJ_close', 'EWU_close', 'EWZ_close', 'EWW_close', 'EZU_close', 'IWM_close', 'MCHI_close','XLK_close', 'XLV_close', 'XLP_close', 'ARKK_close', 'ICLN_close', 'ROBO_close']

# Display sample data
df_portreo

,SPY_close,EWA_close,EWC_close,EWJ_close,EWU_close,EWZ_close,EWW_close,EZU_close,IWM_close,MCHI_close,XLK_close,XLV_close,XLP_close,ARKK_close,ICLN_close,ROBO_close
timestamp,,,,,,,,,,,,,,,,
2022-08-19 04:00:00+00:00,422.14,22.60,35.53,55.63,30.55,31.17,47.50,37.42,194.65,48.47,148.11,132.92,76.91,44.69,22.54,50.43
2022-08-22 04:00:00+00:00,413.35,22.37,35.16,55.06,30.30,30.97,47.07,36.29,190.64,48.66,143.98,131.18,76.13,43.61,22.19,48.82
2022-08-23 04:00:00+00:00,412.35,22.42,35.44,55.19,30.34,32.10,47.37,36.33,190.95,48.75,143.60,129.35,75.80,43.37,22.23,48.85
2022-08-24 04:00:00+00:00,413.67,22.48,35.41,55.17,30.18,32.06,47.12,36.45,192.41,48.75,143.69,129.51,75.99,44.50,22.67,49.24
2022-08-25 04:00:00+00:00,419.51,22.87,35.82,55.80,30.44,32.21,47.63,36.84,195.40,50.79,146.12,130.94,76.34,45.78,23.14,49.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-13 05:00:00+00:00,494.08,22.98,35.29,66.74,31.68,32.27,66.55,46.72,194.61,37.45,202.76,142.86,72.84,47.88,14.12,54.78
2024-02-14 05:00:00+00:00,498.57,23.37,35.92,67.00,32.03,32.55,67.32,47.29,199.13,38.09,204.88,144.10,72.70,50.50,14.51,55.82
2024-02-15 05:00:00+00:00,502.01,23.75,36.70,67.64,32.37,32.80,67.58,47.88,204.44,38.19,204.45,145.15,73.17,51.64,14.73,56.38


In [ ]:
# Jarom SGD Regressor Cell
#Define Features and target

X = df_portreo.drop(columns='SPY_close')
y = df_portreo['SPY_close']

#Scale Data
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler()
scaler.fit(X_train)  
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
#Define model and parmeters
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
SGDmodel = SGDRegressor(max_iter = int(np.ceil(10**6 // len('SPY_close'))), random_state=1)  

#Fit Model
SGDmodel.fit(X_train, y_train)

#Make Predictions
SGDpredictions = SGDmodel.predict(X_test)

score = SGDmodel.score(X_train, y_train)
mse = mean_squared_error(y_test, SGDpredictions)

print(f"R Squared is {score}")
print("Mean Squared Error: ", mse)
print("RMSE: ", mse**(1/2.0))

In [ ]:
#Now that we have the model set, we can run the whole thing for next period predictions
#Fit Model
predict_df = pd.DataFrame(X, columns=['EWA_close', 'EWC_close', 'EWJ_close', 'EWU_close', 'EWZ_close', 'EWW_close', 'EZU_close', 'IWM_close', 'MCHI_close','XLK_close', 'XLV_close', 'XLP_close', 'ARKK_close', 'ICLN_close', 'ROBO_close'])
last_rows = predict_df.tail(1)
last_rows = scaler.transform(last_rows)
SGDnext = SGDmodel.predict(last_rows)
SGD_final = (SGDnext-df_portreo['SPY_close'].tail(1))/df_portreo['SPY_close'].tail(1)
SGD_final

#Set logic for signal
if SGDnext > 0.005:
    SGDprediction = 1
elif SGDnext < -0.005:
    SGDprediction = -1
elif -0.005 <= SGDnext <= 0.005:
    SGDprediction = 0
    
SGDprediction

In [ ]:
#Daniela Prophet 

'''
#14
import pandas as pd
import holoviews as hv
from prophet import Prophet
import hvplot.pandas
import seaborn as sns
from datetime import datetime, timezone
%matplotlib inline

interim = df_portreo.copy()
interim.reset_index(inplace=True)
prophet_model = interim[['timestamp', 'SPY_close']]
prophet_model['timestamp'] = prophet_model['timestamp'].apply(lambda d: d.replace(tzinfo=None))
prophet_model.rename(columns={"timestamp":"ds", "SPY_close":"y"}, inplace=True)

#display(prophet_model)


#Plotting Trends Dataframe'

display(prophet_model.hvplot(title= "Trends", xlabel="Date",ylabel="Closing Prices", xticks="symbol"))


Prophet_trends=Prophet()

Prophet_trends.fit(prophet_model)

Prophet_trends_forecast = Prophet_trends.make_future_dataframe(periods=365, freq='D')
#Prophet_trends_forecast
forecast = Prophet_trends.predict(Prophet_trends_forecast)
forecast

forecast[['yhat', 'yhat_lower', 'yhat_upper']].hvplot()
fig2 = Prophet_trends.plot_components(forecast)
'''